In [1]:
import os, sys
sys.path.append("motion_generation")
sys.path.append("rig_agnostic_encoding/functions")
sys.path.append("rig_agnostic_encoding/models")

from motion_generation.MoE import MoE
import motion_generation
from motion_generation.GRU import GRU
from motion_generation.LSTM import LSTM
from motion_generation.MotionGeneration import MotionGenerationModel
from motion_generation.MotionGeneration_v2 import MotionGenerationModel as MotionGenerationModel_v2
from motion_generation.MotionGeneration_v3 import MotionGenerationModel as MotionGenerationModel_v3
from motion_generation.MotionGenerationRNN import MotionGenerationModelRNN
from motion_generation.MotionGenerationBatch import MotionGenerationModelBatch
from rig_agnostic_encoding.models.MLP import MLP
from rig_agnostic_encoding.models.MLP_MIX import MLP_MIX
from rig_agnostic_encoding.models.VAE import VAE
from rig_agnostic_encoding.functions.DataProcessingFunctions import clean_checkpoints
from GlobalSettings import MODEL_PATH
import bz2

import torch
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import func as F
import _pickle as pickle
import json as js
import importlib

In [2]:
config = {
    "hidden_dim": 512,
    "k": 256,
    "lr": 1e-4,
    "batch_size": 32,
    "keep_prob": .2,
    "loss_fn":torch.nn.functional.mse_loss,
    "optimizer":torch.optim.Adam,
    "scheduler":torch.optim.lr_scheduler.StepLR,
    "scheduler_param": {"step_size":80, "gamma":.9},
    "basis_func":"gaussian",
    "n_centroid":128,
    "k_experts": 4,
    "gate_size": 128,
    "g_hidden_dim": 512,
    "num_layers": 4,
    "autoregress_prob":0,
    "autoregress_inc":.2,
    "autoregress_ep":50,
    "autoregress_max_prob":.5,
    "cost_hidden_dim":128,
    "seq_len":13,
    "device":"cuda"
    }

In [6]:
MAX_FILES = -1
data_path = "/home/nuoc/Documents/MEX/data/Dataset_R1_One_1"
file_paths = []
for dname, dirs, files in os.walk(data_path):
    for i, file in enumerate(files):
        file_paths.append(os.path.join(dname, file))
        if MAX_FILES > 0 and i >= MAX_FILES:
            break

In [7]:
phase_features = ["phase_vec_l2"]
pose_features = ["pos", "rotMat2", "velocity"]
cost_features = ["posCost", "rotCost"]
features = phase_features + pose_features + cost_features
clips = []
feature_dims = {}

In [8]:
data = F.process_data_multithread(file_paths, features, use_window=False)
feature_dims = data[0][1]
clips = [np.copy(d[0]) for d in data]


2021-05-04 15:37:24,269	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265


In [9]:
phase_dim = sum([feature_dims[feature] for feature in phase_features])
pose_dim = sum([feature_dims[feature] for feature in pose_features])
cost_dim = sum([feature_dims[feature] for feature in cost_features])
print(phase_dim, " ", pose_dim, " ", cost_dim)

8   372   24


In [10]:
x_tensors = torch.stack([F.normaliseT(torch.from_numpy(clip[:-1])).float() for clip in clips])
y_tensors = torch.stack([torch.from_numpy(clip[1:]).float() for clip in clips])

dataset = TensorDataset(torch.Tensor(x_tensors), torch.Tensor(y_tensors))
N = len(x_tensors)

train_ratio = int(.7*N)
val_ratio = int((N-train_ratio) / 2.0)
test_ratio = N - train_ratio - val_ratio
train_set, val_set, test_set = random_split(dataset, [train_ratio, val_ratio, test_ratio], generator=torch.Generator().manual_seed(2021))
print(len(train_set), len(val_set), len(test_set))

168 36 36


In [6]:
dataset_name = "R1_MoGenData_nonNorm_phaseKey"

In [12]:
data = {"data":[train_set, val_set, test_set],
        "feature_dims":feature_dims,
        "dims":[phase_dim, pose_dim, cost_dim]}
F.save(data, dataset_name, "/home/nuoc/Documents/MEX/data")

In [7]:
obj = F.load("/home/nuoc/Documents/MEX/data/"+dataset_name+".pbz2")
train_set, val_set, test_set = obj["data"]
feature_dims = obj["feature_dims"]
phase_dim, pose_dim, cost_dim= obj["dims"]

In [8]:
print(len(train_set), train_set[0][0].shape)
print(len(val_set), val_set[0][0].shape)
print(len(test_set), test_set[0][0].shape)

168 torch.Size([299, 404])
36 torch.Size([299, 404])
36 torch.Size([299, 404])


In [9]:
model_name = "MLP_GRU_R1_One_1_Full"

feature_dims2 = {
    "phase_dim": phase_dim,
    "pose_dim": pose_dim,
    "cost_dim": cost_dim,
    "g_input_dim": config["k"] + config["cost_hidden_dim"],
    "g_output_dim":phase_dim + config["k"] + cost_dim
    }

in_slice = [phase_dim, pose_dim, cost_dim]
out_slice = [phase_dim, config["k"], cost_dim]

MLPMIX = MLP_MIX(config=config, input_dims=[pose_dim])
pose_encoder = MLPMIX.active_models[0]
middle_layer = MLPMIX.cluster_model

model = MotionGenerationModelRNN(config=config, Model=GRU, pose_autoencoder=pose_encoder, middle_layer=middle_layer,
                                 feature_dims=feature_dims2,
                                 input_slicers=in_slice, output_slicers=out_slice,
                                 train_set=train_set, val_set=val_set, test_set=test_set,
                                 name=model_name
                                   )

In [10]:
MAX_EPOCHS = 300

checkpoint_callback = ModelCheckpoint(monitor="avg_val_loss", save_top_k=3)
earlystopping = EarlyStopping(monitor="avg_val_loss", patience=20)
logger=TensorBoardLogger(save_dir="logs/", name=model_name, version="0.1")

trainer = pl.Trainer(
    default_root_dir="/home/nuoc/Documents/MEX/src/motion_generation/checkpoints",
    gpus=1, precision=16,
    callbacks=[earlystopping],
    min_epochs=20,
    logger=logger,
    max_epochs=MAX_EPOCHS,
    stochastic_weight_avg=True
)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
Using native 16bit precision.


In [14]:
model.workers = 6
model.seq_len = config["seq_len"]

In [15]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type       | Params
------------------------------------------------
0 | pose_autoencoder | MLP        | 1.2 M 
1 | middle_layer     | Sequential | 65.8 K
2 | cost_encoder     | MLP        | 36.2 K
3 | generationModel  | GRU        | 6.3 M 
------------------------------------------------
7.5 M     Trainable params
0         Non-trainable params
7.5 M     Total params
30.157    Total estimated model params size (MB)


1

In [16]:
trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.03913438692688942, 'test_loss': 0.062133900821208954}
--------------------------------------------------------------------------------


[{'test_loss': 0.062133900821208954, 'ptl/test_loss': 0.03913438692688942}]

In [17]:
clean_checkpoints(path=os.path.join(MODEL_PATH,model_name))
# clean_checkpoints(path="/home/nuoc/Documents/MEX/models/version_0.2/MLP_MoE_R1_One_1_Full_v3_smooth_loss")

In [61]:
# model2 = MotionGenerationModel_v2.load_checkpoint("
filename = "/home/nuoc/Documents/MEX/models/version_0.2/MLP_MoE_R1_One_1_Full_TUNE/0.06325527280569077.pbz2"
with bz2.BZ2File(filename, "rb") as f:
    obj = pickle.load(f)

pose_autoencoder = MLP.load_checkpoint(obj["pose_autoencoder_path"])
cost_encoder = MLP.load_checkpoint(obj["cost_encoder_path"])
generationModel = MoE.load_checkpoint(obj["motionGenerationModelPath"])

model = MotionGenerationModel_v2(config=obj["config"], feature_dims=obj["feature_dims"], Model=MoE,
                              input_slicers=obj["in_slices"], output_slicers=obj["out_slices"],
                              name=obj["name"])

model.in_slices = obj["in_slices"]
model.out_slices = obj["out_slices"]
model.pose_autoencoder = pose_autoencoder
model.cost_encoder = cost_encoder
model.generationModel = generationModel

In [64]:
model.test_set = test_set

[0, 62, 434, 458]
MLP(
  (encoder): Sequential(
    (fc0): Linear(in_features=372, out_features=512, bias=True)
    (act0): ELU(alpha=1.0)
    (drop): Dropout(p=0.2, inplace=False)
    (fc1): Linear(in_features=512, out_features=512, bias=True)
    (act1): ELU(alpha=1.0)
    (fc2): Linear(in_features=512, out_features=256, bias=True)
  )
  (decoder): Sequential(
    (fc0): Linear(in_features=256, out_features=512, bias=True)
    (act0): ELU(alpha=1.0)
    (drop): Dropout(p=0.2, inplace=False)
    (fc1): Linear(in_features=512, out_features=512, bias=True)
    (act1): ELU(alpha=1.0)
    (fc2): Linear(in_features=512, out_features=372, bias=True)
  )
)
MLP(
  (encoder): Sequential(
    (fc0): Linear(in_features=24, out_features=128, bias=True)
    (act0): ELU(alpha=1.0)
    (drop): Dropout(p=0.2, inplace=False)
    (fc1): Linear(in_features=128, out_features=128, bias=True)
    (act1): ELU(alpha=1.0)
    (fc2): Linear(in_features=128, out_features=128, bias=True)
  )
  (decoder): Sequent

In [9]:
model.autoregress_prob = 0
trainer.test(model)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.020404554903507233, 'test_loss': 0.038231831043958664}
--------------------------------------------------------------------------------


[{'test_loss': 0.038231831043958664, 'ptl/test_loss': 0.020404554903507233}]

In [10]:
model.autoregress_prob = 1
trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'ptl/test_loss': 0.027046212926506996, 'test_loss': 0.04136933758854866}
--------------------------------------------------------------------------------


[{'test_loss': 0.04136933758854866, 'ptl/test_loss': 0.027046212926506996}]

In [18]:
# model.autoregress_prob = 0

n = 5
idx = np.random.randint(0, len(test_set), n)
original = []
generated = []
# pose_idx_upper = feature_dims2["phase_dim"] + feature_dims["pos"] + feature_dims["rotMat2"]
pose_idx_upper = model.in_slices[1] + feature_dims["pos"] + feature_dims["rotMat2"]
print(pose_idx_upper)



287


In [34]:
model.autoregress_prob = 1
with torch.no_grad():
    model.eval()
    model.cpu()
    # for i in range(1):
    # original =
    # x = x_tensors[idx]
    x = torch.stack([test_set[i][0] for i in idx])
    y = torch.stack([test_set[i][1] for i in idx])
    shape = x.shape
    x = x.view(-1, shape[-1])
    x.to("cpu")
    model.generationModel.reset_hidden(batch_size=x.shape[0])
    model.generationModel.hidden = model.generationModel.hidden.cpu()
        # size = x.size()[1]
    # g_frames = []
        # for i in range(0, size):
        # x_c = x[:, i, :]
    out = torch.cat(model(x), dim=1).view(shape)
            # x_c = torch.cat(out, dim=1)
    # g_frames.append(x_c.unsqueeze(1))
        # original.append(o_frames)
    # generated.append(torch.cat(g_frames, dim=1))
    generated = out

In [35]:
print(y.size())
# generated = generated.view(-1, 298, generated.shape[-1])
print(generated.size())

torch.Size([5, 299, 404])
torch.Size([5, 299, 404])


In [36]:
phase_dim = feature_dims2["phase_dim"]
toPosDim = phase_dim+feature_dims["pos"]
toRotDim = toPosDim + feature_dims["rotMat2"]

gPos = generated[:, :, phase_dim:toPosDim]
gRot = generated[:, :, toPosDim:toRotDim]

oPos = y[:, :, phase_dim:toPosDim]
oRot = y[:, :, toPosDim:toRotDim]

print(gPos.shape, gRot.shape)
print(oPos.shape, oRot.shape)


torch.Size([5, 299, 93]) torch.Size([5, 299, 186])
torch.Size([5, 299, 93]) torch.Size([5, 299, 186])


In [37]:
clip_length = gPos.shape[1]
gPos_r = gPos.reshape((n, clip_length, -1, 3))
gRot_r = gRot.reshape((n, clip_length, -1, 3, 2))
oPos_r = oPos.reshape((n, clip_length, -1, 3))
oRot_r = oRot.reshape((n, clip_length, -1, 3, 2))

print("Pos loss: ", torch.nn.functional.mse_loss(gPos_r, oPos_r))
print("Rot loss: ", torch.nn.functional.mse_loss(gRot_r, oRot_r))

Pos loss:  tensor(0.0052)
Rot loss:  tensor(0.0370)


In [38]:
template = js.load(open("/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/False_2_0.json"))

In [39]:
def insert_pos(positions, rotations, name="Replay"):
    shape = positions.shape
    for c in range(shape[0]):
        for f in range(shape[1]):
            for j in range(shape[2]):
                template["frames"][f]["joints"][j]["position"]["x"] = positions[c,f,j,0].item()
                template["frames"][f]["joints"][j]["position"]["y"] = positions[c,f,j,1].item()
                template["frames"][f]["joints"][j]["position"]["z"] = positions[c,f,j,2].item()

                for r, cell in enumerate(["x", "y", "z"]):
                    for col, column in enumerate(["c0", "c1"]):
                        template["frames"][f]["joints"][j]["rotMat"][column][cell] = rotations[c,f,j,r, col].item()
        with open("{}_{}.json".format(name, c), "w") as f:
            js.dump(template, f)

insert_pos(oPos_r, oRot_r, "/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/Replay_original_GRU")
insert_pos(gPos_r, gRot_r, "/home/nuoc/.config/unity3d/DefaultCompany/Procedural Animation/TestAll_1_R1_One_1/Replay_generated_GRU")